# Import packages

In [ ]:
import numpy as np
import numpy.random as npr
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.linalg import orth
import time

from sca.models import SCA, WeightedPCA
from sca.util import get_sample_weights, get_accuracy

In [ ]:
# %load_ext autoreload
# %autoreload 2

# Simulate data
Data is simulated from sparsely occurring latents

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Parameters
n_pad = 200  # Padding before and after
conditions = [
    (100, 200, 400, 1.0 + 0.02, 1.0),      # Condition 1
    (100, 100, 400, 1.0 + 0.02, 1.0),      # Condition 2
    (200, 200, 400, -1.0 - 0.02, -1.0),    # Condition 3
]

# Helper function to generate each condition block
def create_condition(f1_dur, cooccur_dur, f2_dur, f1_val, f2_val):
    f1_only = np.column_stack([np.full(f1_dur, f1_val), np.zeros(f1_dur)])
    cooccur = np.column_stack([np.full(cooccur_dur, f1_val), np.full(cooccur_dur, f2_val)])
    f2_only = np.column_stack([np.zeros(f2_dur), np.full(f2_dur, f2_val)])
    return np.vstack([f1_only, cooccur, f2_only])

# Build all condition segments with padding
all_segments = []
for (f1_dur, cooccur_dur, f2_dur, f1_val, f2_val) in conditions:
    cond = create_condition(f1_dur, cooccur_dur, f2_dur, f1_val, f2_val)
    pad_before = np.zeros((n_pad, 2))
    pad_after = np.zeros(((1000-f1_dur-cooccur_dur-f2_dur), 2))
    segment = np.vstack([pad_before, cond, pad_after])
    all_segments.append(segment)

# Plot conditions in separate subplots (stacked vertically)
fig, axes = plt.subplots(len(all_segments), 1, figsize=(10, 6), sharex=False)

for i, segment in enumerate(all_segments):
    time = np.arange(segment.shape[0])
    axes[i].plot(time, segment[:, 0], label="Factor 1",)
    axes[i].plot(time, segment[:, 1], label="Factor 2")


axes[-1].set_xlabel("Time")
plt.tight_layout()
plt.savefig('sim1_ground_truth.pdf')
plt.show()

In [ ]:
# Parameters
n_samples = 1000              # Samples per condition
n_pad = 400                   # Padding before and after

# Define condition blocks
cond1 = np.column_stack([np.ones(n_samples)*1.5+0.02, np.zeros(n_samples)])     # [1, 0]
cond2 = np.column_stack([np.zeros(n_samples), np.ones(n_samples)*2])            # [0, 1]
cond3 = np.column_stack([np.ones(n_samples)+0.02, np.ones(n_samples)])          # [1, 1]

conditions = [cond1, cond2, cond3]

# Build each condition segment with padding
all_segments = []
for cond in conditions:
    pad_before = np.zeros((n_pad, 2))
    pad_after = np.zeros((n_pad, 2))
    segment = np.vstack([pad_before, cond, pad_after])
    all_segments.append(segment)

# Plot conditions in separate subplots (stacked vertically)
fig, axes = plt.subplots(len(all_segments), 1, figsize=(10, 6), sharex=False)

for i, segment in enumerate(all_segments):
    time = np.arange(segment.shape[0])
    axes[i].set_ylim(-0.1, 2.1)
    axes[i].plot(time, segment[:, 0], label="Factor 1")
    axes[i].plot(time, segment[:, 1], label="Factor 2")

axes[-1].set_xlabel("Time")
plt.tight_layout()
plt.savefig('sim2_ground_truth.pdf')
plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Parameters
# factor1_duration = 1000
# factor2_duration = 500
# n_pad = 400  # Padding before and after

# # Define condition blocks
# # Single factor conditions
# i = 3
# cond1 = np.column_stack([np.ones(factor1_duration)+0.02, np.zeros(factor1_duration)])  # Factor 1 active
# cond2 = np.column_stack([np.zeros(i*factor2_duration), np.ones(i*factor2_duration)])       # Factor 2 active

# # Dual factor condition (pad factor 2 to match factor 1's length)
# factor1 = np.ones(factor1_duration) + 0.02
# factor2 = np.concatenate([np.ones(factor2_duration), np.zeros(factor1_duration - factor2_duration)])
# cond3 = np.column_stack([factor1, factor2])

# cond4 = np.column_stack([-np.ones(factor1_duration)+0.02, np.zeros(factor1_duration)])  # Factor 1 active
# cond5 = np.column_stack([np.zeros(i*factor2_duration), -np.ones(i*factor2_duration)])       # Factor 2 active

# # factor1 = -np.ones(factor1_duration) + 0.02
# # factor2 = np.concatenate([-np.ones(factor2_duration), np.zeros(factor1_duration - factor2_duration)])
# # cond6 = np.column_stack([factor1, factor2])

# # Combine with padding
# all_segments = []
# segment_starts = []

# current_index = 0
# # for cond in [cond1, cond2, cond3]:
# for cond in [cond1, cond2, cond3, cond4, cond5]:
#     pad_before = np.zeros((n_pad, 2))
#     pad_after = np.zeros((n_pad, 2))
#     segment = np.vstack([pad_before, cond, pad_after])
#     all_segments.append(segment)
#     segment_starts.append(current_index)
#     current_index += segment.shape[0]

# # Final data array
# Z = np.vstack(all_segments)
# time = np.arange(Z.shape[0])

# # Plot
# plt.figure(figsize=(12, 5))
# plt.plot(time, Z[:, 0], label="Factor 1", color='blue')
# plt.plot(time, Z[:, 1], label="Factor 2", color='green')

# # Draw vertical lines at segment starts (except the first one)
# for idx in segment_starts[1:]:
#     plt.axvline(idx, color='gray', linestyle='--')

# # Final styling
# plt.axhline(0, color='black', linewidth=0.5)
# plt.xlabel("Time")
# plt.ylabel("Factor")
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
Z = np.vstack(all_segments)
np.random.seed(0) #To get the same simulated data

N_neurons=50 #Number of neurons
R_sim=2 #Number of dimensions in lowD representations

#Orthogonal matrix that projects low dimensional space to full neural space
V_tmp=orth(npr.randn(R_sim,N_neurons).T).T 

#Create high-dimensional neural activity    
b=npr.randn(N_neurons) #Offset of neurons
X0=Z@V_tmp[:R_sim,:]+b #Project into high-dimensional space and add offset
noise_level = 0.1
X0=X0+noise_level*npr.randn(X0.shape[0],X0.shape[1]) #Add noise

In [ ]:

# Example neurons to plot
example_neurons = [0, 5]   # pick any two neuron indices
n_conditions = 3

# Split time into conditions
n_time_total, n_neurons = X0.shape
T = n_time_total // n_conditions   # samples per condition

plt.figure(figsize=(9, 6))

for row, f_idx in enumerate(example_neurons):       # rows = factors
    for col in range(n_conditions):                 # cols = conditions
        start = col * T
        end = (col + 1) * T
        trace = X0[start:end, f_idx]

        ax = plt.subplot(len(example_neurons), n_conditions, row*n_conditions + col + 1)
        ax.plot(trace,color  = "#6e0f3e")
        ax.set_yticks([])

        # Labels
        if row == len(example_neurons) - 1:   # bottom row x-labels
            ax.set_xlabel("Time")
        else:
            ax.set_xticks([])
        if col == 0:                          # left col y-labels
            ax.set_ylabel(f"Factor {f_idx+1}")
        if row == 0:                          # top row titles
            ax.set_title(f"Cond {col+1}")

plt.savefig('sim1_single_neurons.pdf')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
for i in range(R_sim):
    
    #Plot ground truth
    plt.subplot(R_sim,2,2*i+1)
    plt.plot((Z)[:,i]) 
    
    plt.ylim([-1.1, 1.1])
    plt.yticks([])
    if i<R_sim-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')
    
    # Plot Example neurons
    plt.subplot(R_sim,2,2*i+2)
    plt.plot((X0)[:,i]) 
    
#     plt.ylim([-1.1, 1.1])
    plt.yticks([])    
    if i<R_sim-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')

# Titles
plt.subplot(R_sim,2,1)
plt.title('Simulated latents')

plt.subplot(R_sim,2,2)
plt.title('Example simulated "neurons" ')


# Preprocess data (optional)

We have found that the method usually works better when zero-centering the data.

In this specific example, if you don't zero-center the data, it will take ~10000 iterations to converge to the ground truth, rather than ~2000.

In [ ]:
# X = np.copy(X0)
X=np.copy(X0-np.mean(X0,axis=0)[None,:])

# Set required model parameters

In [ ]:
#Number of dimensions in the low-D model you're fitting
n_components=2

# Set some optional model parameters

All of these have default values, so it's not essential to set them. Below just shows most of the parameters you can set.

In [ ]:
#Number of epochs of model fitting (default is 3000)
n_epochs=3000

#Learning rate of model fitting (default is .001)
lr=.001

#Whether to have a strict orthogonality constraint in the loadings (default is False)
#Note that the version of SCA that has orth=False runs faster
# orth=False

#Initialization of weights - can be 'pca' or 'rand' (default is 'pca')
init='pca'

#We would recommend using the default lambda hyperparameters, at least to start. 
#When running SCA, it will print what the default values are for the given dataset

#Strength of the sparsity penalty
lam_sparse=.1

#Strength of the orthogonality penality (Note - This is only used in the version without a hard orthogonality constraint)
lam_orthog=1

In [ ]:
#How much to weight each data point in time
#(this can be helpful for making sure dimensions still aim to explain time points with low activity)

sample_weights=np.ones([X.shape[0],1]) #Weight equally

# sample_weights=get_sample_weights(X) #Weight inversely to norm of activity at each time point

# Fit Weighted PCA Model (for comparison)

In [ ]:
#Fit weighted PCA
#Note that this function does not automatically subtract the mean from the data (as in many PCA functions)

#Decleare model
wpca=WeightedPCA(n_components=n_components,rotate=False)
wpca_vari=WeightedPCA(n_components=n_components,rotate=True)

#Fit and get the low dimensional representation (the principal components)
#Note that the sample_weight input is optional and will default to no sample weighting
pca_latent = wpca.fit_transform(X,sample_weight=sample_weights)
pca_vari_latent = wpca_vari.fit_transform(X,sample_weight=sample_weights)

# Fit ICA

In [ ]:
X_weighted = X * sample_weights

In [ ]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=n_components)
ica_latent = ica.fit_transform(X)     # Independent components


# Fit SCA Model
We show options for running SCA below (with and without optional parameters).

In [ ]:
#Declare SCA model without all the optional parameters
# sca=SCA(n_components=n_components,n_epochs=6000,lam_sparse = .1, init='rand')
sca=SCA(n_components=n_components)

#Declare SCA model with the optional parameters
# sca=SCA(n_components=n_components,orth=orth, lam_sparse=lam_sparse, lam_orthog=lam_orthog, lr=lr,n_epochs=n_epochs, init=init)

 
#Fit the model and get the low dimensional representation
#Note that the sample_weight input is optional and will default to no sample weighting
sca_latent=sca.fit_transform(X=X, sample_weight=sample_weights)

In [ ]:
#Plot the loss over all iterations
plt.figure()
plt.plot(sca.losses)
plt.xlabel('Training Epoch')
plt.ylabel('Loss')
plt.title('Loss over training')

#Plot the loss over the last 100 iterations (to see if it has truly hit a plateau)
# plt.figure()
# plt.plot(losses[-100:-1])
# plt.xlabel('Training Epoch')
# plt.ylabel('Loss')

# Plot Latents

### Plot unordered lowD representations

In [ ]:
#Ground truth
# T = 3300
# Z_extra=np.zeros([T,n_components])
# Z_extra[:,:R_sim]=Z

Z_extra=Z

plt.figure(figsize=(20, 5))  # Wider figure for 4 columns

for i in range(n_components):
    
    # --- Plot Ground Truth ---
    plt.subplot(n_components, 5, 5 * i + 1)
    plt.plot(Z_extra[:, i])
    # plt.ylim([-1.6, 1.6])
    if i < n_components - 1:
        plt.xticks([])
    else:
        plt.xlabel('Time')
    
    # --- Plot SCA ---
    plt.subplot(n_components, 5, 5 * i + 2)
    plt.plot(sca_latent[:, i])
    # plt.ylim([-1.6, 1.6])
    if i < n_components - 1:
        plt.xticks([])
    else:
        plt.xlabel('Time')

    # --- Plot PCA ---
    plt.subplot(n_components, 5, 5 * i + 3)
    plt.plot(pca_latent[:, i])
    # plt.ylim([-1.6, 1.6])
    if i < n_components - 1:
        plt.xticks([])
    else:
        plt.xlabel('Time')

    # --- Plot PCA+Varimax ---
    plt.subplot(n_components, 5, 5 * i + 4)
    plt.plot(pca_vari_latent[:, i])
    # plt.ylim([-1.6, 1.6])
    if i < n_components - 1:
        plt.xticks([])
    else:
        plt.xlabel('Time')    

    # --- Plot ICA ---
    plt.subplot(n_components, 5, 5 * i + 5)
    plt.plot(ica_latent[:, i])
    # plt.ylim([-1.6, 1.6])
    if i < n_components - 1:
        plt.xticks([])
    else:
        plt.xlabel('Time')

# Titles
plt.subplot(n_components, 5, 1)
plt.title('True LowD Projections')

plt.subplot(n_components, 5, 2)
plt.title('SCA LowD Projections')

plt.subplot(n_components, 5, 3)
plt.title('PCA LowD Projections')

plt.subplot(n_components, 5, 4)
plt.title('PCA+Varimax LowD Projections')

plt.subplot(n_components, 5, 5)
plt.title('ICA LowD Projections')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

corr = np.corrcoef(sca_latent.T, Z.T)[:2, 2:]  # shape (2,2)

# Step 2: find best matching order
order = np.argmax(np.abs(corr), axis=0)   # which PCA latent matches each factor
signs = np.sign(corr[order, range(2)])    # sign of that correlation

# Step 3: reorder and flip PCA latents
sca_aligned = np.zeros_like(sca_latent)
for j in range(2):  # for each factor
    sca_aligned[:, j] = sca_latent[:, order[j]] * signs[j]

# Example neurons to plot
example_factors = [0, 1]   # pick any two neuron indices
n_conditions = 3

# Split time into conditions
n_time_total, n_neurons = sca_latent.shape
T = n_time_total // n_conditions   # samples per condition

# plt.figure(figsize=(9, 6))
fig, axes = plt.subplots(
    n_neurons, n_conditions,
    figsize=(9, 6),
    # sharex=True, sharey=True   # makes all panels share axes
)

for row, f_idx in enumerate(example_factors):       # rows = factors
    for col in range(n_conditions):                 # cols = conditions
        start = col * T
        end = (col + 1) * T
        trace = sca_aligned[start:end, f_idx]
        
        ax = axes[row, col]

        ax = plt.subplot(len(example_factors), n_conditions, row*n_conditions + col + 1)
        ax.plot(trace,color  = "#6e0f3e")
        ax.set_yticks([])

        # Labels
        if row == len(example_factors) - 1:   # bottom row x-labels
            ax.set_xlabel("Time")
        else:
            ax.set_xticks([])
        if col == 0:                          # left col y-labels
            ax.set_ylabel(f"Factor {f_idx+1}")
        if row == 0:                          # top row titles
            ax.set_title(f"Cond {col+1}")
plt.savefig('sim1_SCA_factors.pdf')
plt.tight_layout()
plt.show()




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

corr = np.corrcoef(pca_latent.T, Z.T)[:2, 2:]  # shape (2,2)

# Step 2: find best matching order
order = np.argmax(np.abs(corr), axis=0)   # which PCA latent matches each factor
signs = np.sign(corr[order, range(2)])    # sign of that correlation

# Step 3: reorder and flip PCA latents
pca_aligned = np.zeros_like(pca_latent)
for j in range(2):  # for each factor
    pca_aligned[:, j] = pca_latent[:, order[j]] * signs[j]

# Example neurons to plot
example_factors = [0, 1]   # pick any two neuron indices
n_conditions = 3

# Split time into conditions
n_time_total, n_neurons = pca_latent.shape
T = n_time_total // n_conditions   # samples per condition

# plt.figure(figsize=(9, 6))
fig, axes = plt.subplots(
    n_neurons, n_conditions,
    figsize=(9, 6),
    # sharex=True, sharey=True   # makes all panels share axes
)

for row, f_idx in enumerate(example_factors):       # rows = factors
    for col in range(n_conditions):                 # cols = conditions
        start = col * T
        end = (col + 1) * T
        trace = pca_aligned[start:end, f_idx]

        ax = axes[row, col]
        
        ax = plt.subplot(len(example_factors), n_conditions, row*n_conditions + col + 1)
        ax.plot(trace,color  = "#6e0f3e")
        ax.set_yticks([])

        # Labels
        if row == len(example_factors) - 1:   # bottom row x-labels
            ax.set_xlabel("Time")
        else:
            ax.set_xticks([])
        if col == 0:                          # left col y-labels
            ax.set_ylabel(f"Factor {f_idx+1}")
        if row == 0:                          # top row titles
            ax.set_title(f"Cond {col+1}")

plt.savefig('sim1_PCA_factors.pdf')
plt.tight_layout()
plt.show()


### Order low-dimensional representations by time of maximum variance explained by that dimension.

In [ ]:
#Amount of squared activity each dimension explains in PCA
infs_pca=[np.sum((pca_latent[:,i:i+1]@wpca.params['V'][i:i+1,:])**2,axis=1) for i in range(n_components)]

#Amount of squared activity each dimension explains in SCA
infs_sca=[np.sum((sca_latent[:,i:i+1]@sca.params['V'][:,i:i+1].T)**2,axis=1) for i in range(n_components)]

infs_ica = [
    np.sum((ica_latent[:, i:i+1] @ ica.mixing_[:, i:i+1].T)**2, axis=1)
    for i in range(n_components)
]

#Find the time point of each dimension that has the largest squared activity explained
max_array_pca=[np.argmax(infs_pca[i]) for i in range(n_components)]
max_array_sca=[np.argmax(infs_sca[i]) for i in range(n_components)]
max_array_ica=[np.argmax(infs_ica[i]) for i in range(n_components)]


#Order dimensions
pca_order=np.argsort(np.array(max_array_pca))
sca_order=np.argsort(np.array(max_array_sca))
ica_order=np.argsort(np.array(max_array_ica))


#### Plot!

In [ ]:
#Ground truth
Z_extra=np.zeros([T,n_components])
Z_extra[:,:R_sim]=Z


plt.figure(figsize=(15,5))
for i in range(n_components):
    
    #Plot ground truth
    plt.subplot(n_components,3,3*i+1)
    plt.plot((Z_extra)[:,i]) 
    
    plt.ylim([-1.6, 1.6])
    # plt.yticks([])
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')
    
    # Plot SCA results
    plt.subplot(n_components,3,3*i+2)
    plt.plot(sca_latent[:,sca_order[i]])
    
    plt.ylim([-1.6, 1.6])
    # plt.yticks([])    
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')

    # Plot PCA results
    plt.subplot(n_components,3,3*i+3)
    plt.plot(pca_latent[:,pca_order[i]])
    
    plt.ylim([-1.6, 1.6])
    # plt.yticks([])
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')        

#Titles
plt.subplot(n_components,3,1)
plt.title('True LowD Projections')

plt.subplot(n_components,3,2)
plt.title('SCA LowD Projections')

plt.subplot(n_components,3,3)
plt.title('PCA LowD Projections')


In [ ]:
plt.figure(figsize=(20,5))
for i in range(n_components):
    
    #Plot ground truth
    plt.subplot(n_components,4,4*i+1)
    plt.plot((Z_extra)[:,i]) 
    
    plt.ylim([-1.6, 1.6])
    # plt.yticks([])
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')
    
    # Plot SCA results
    plt.subplot(n_components,4,4*i+2)
    plt.plot(sca_latent[:,sca_order[i]])
    
    plt.ylim([-1.6, 1.6])
    # plt.yticks([])    
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')

    # Plot PCA results
    plt.subplot(n_components,4,4*i+3)
    plt.plot(pca_latent[:,pca_order[i]])
    
    plt.ylim([-1.6, 1.6])
    # plt.yticks([])
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')        

    # Plot ICA results
    plt.subplot(n_components,4,4*i+4)
    plt.plot(ica_latent[:,ica_order[i]])
    
    # plt.ylim([-1.6, 1.6])
    # plt.yticks([])
    if i<n_components-1:
        plt.xticks([])
    else:
        plt.xlabel('Time')   

#Titles
plt.subplot(n_components,4,1)
plt.title('True LowD Projections')

plt.subplot(n_components,4,2)
plt.title('SCA LowD Projections')

plt.subplot(n_components,4,3)
plt.title('PCA LowD Projections')

plt.subplot(n_components,4,4)
plt.title('ICA LowD Projections')


## Look at how orthogonal the projection is
Show how orthogonal each of the latent dimensions are to each other

In [ ]:
product=sca.params['V']@sca.params['V'].T
plt.imshow(product,clim=[-1,1],cmap='RdBu')
plt.colorbar()

## Get and plot reconstructed data

In [ ]:
## Get reconstructed high-dimensionaldata
Xhat=sca.reconstruct(X)

In [ ]:
## Plot example neurons

plot_neurons=[0,1,2,3,4] #example neurons to plot
n_plot_neurons=len(plot_neurons)

plt.figure(figsize=(6,6))
for i in range(n_plot_neurons):
    plt.subplot(n_plot_neurons,1,i+1)
    plt.plot(X[:,i])
    plt.plot(Xhat[:,i])
    
plt.legend(['Actual','Pred'],loc='right')

## Compare goodness of fit between SCA and PCA model (on the fit data)

Get: <br> 1) **Reconstruction loss** in the cost function (the weighted sum squared error) <br> 2) **R2 value** of the model (Neurons are weighted by their amount of variance, and sample-weighting is used)

In [ ]:
#For SCA, the reconstruction loss and r2 automatically get assigned to the model after fitting as attributes
print('SCA r2:', sca.r2_score)
print('SCA reconst_loss:', sca.reconstruction_loss)

In [ ]:
#For PCA, we use the get_accuracy function from sca.utils
[pca_r2_score, pca_reconstruction_loss]=get_accuracy(wpca,X,sample_weights)

print('PCA r2:', pca_r2_score)
print('PCA reconstr_loss:', pca_reconstruction_loss)

In [ ]:
from sklearn.metrics import r2_score
X_reconstructed = np.dot(ica_latent, ica.mixing_.T) + ica.mean_

ica_reconstruction_loss = np.sum((X - X_reconstructed) ** 2)
ica_r2_score = r2_score(X,X_reconstructed,sample_weight=sample_weights,multioutput='variance_weighted')

print('ICA r2:', ica_r2_score)
print('ICA reconstr_loss:', ica_reconstruction_loss)

## Some other attributes of SCA model

In [ ]:
#Squared neural activity that each latent explains (Note this is slightly different than variance because of the offset term)
sca.explained_squared_activity

In [ ]:
#Hyperparameters of the model
print('n_components:',sca.n_components)
print('lam_sparse:',sca.lam_sparse)
print('lam_orthog:',sca.lam_orthog)

In [ ]:
#parameters of the SCA model (commented out below)
#See Methods of manuscript for further details on the parameters

# sca.params['U']     #Matrix that projects high-d data to low-d space, size [N_neurons , n_components]
# sca.params['b_u']   #Offset for low-d space, size [n_components]
# sca.params['V']     #Matrix that projects low-d data to high-d space, size [n_components, N]
# sca.params['b_v']   #Offset for high-d space, size [N_neurons]